# Lacam integration

In [1]:
import numpy as np
import os
import time
import ctypes

from enum import Enum
from pogema import pogema_v0, GridConfig
from pogema.animation import AnimationMonitor, AnimationConfig
from IPython.display import SVG, display, HTML
from pogema.a_star_policy import h, GridMemory, a_star
from pogema import GridConfig

In [2]:
class LacamLib:
    def __init__(self, lib_path):
        self._lacam_lib = ctypes.CDLL(lib_path)

        self._lacam_lib.run_lacam.argtypes = [
            ctypes.c_char_p,  # map_name
            ctypes.c_char_p,  # scene_name
            ctypes.c_int      # N
        ]
        self._lacam_lib.run_lacam.restype = ctypes.c_char_p

    def run_lacam(self, map_file_content, scene_file_content, num_agents):
        map_file_bytes = map_file_content.encode('utf-8')
        scenario_file_bytes = scene_file_content.encode('utf-8')

        num_agents_int = ctypes.c_int(num_agents)

        result = self._lacam_lib.run_lacam(
            map_file_bytes, 
            scenario_file_bytes, 
            num_agents_int
        )

        try:
            result_str = result.decode('utf-8')
        except Exception as e:
            print(f'Exception occured while running Lacam: {e}')
            raise e
        
        assert result_str != "ERROR"
        return result_str

In [3]:
class LacamAgent:
    def __init__(self, idx, lacam_lib, global_planning, seed=0):
        self._moves = GridConfig().MOVES
        self._reverse_actions = {tuple(self._moves[i]): i for i in range(len(self._moves))}

        self.idx = idx
        self._gm = None
        self._saved_xy = None
        self.clear_state()
        self._rnd = np.random.default_rng(seed)

        self.lacam_lib = lacam_lib
        self.previous_goal = None
        self.calculated_path = []
        self.global_planning = global_planning

    def is_new_goal(self, new_goal):
        return not self.previous_goal == new_goal
    
    def set_new_goal(self, new_goal):
        self.previous_goal = new_goal

    def set_calculated_path(self, new_path):
        # inverse list to easily pop steps from the end
        self.calculated_path = new_path[::-1]

    def get_map_file(self):
        height = self._gm._memory.shape[0]
        width = self._gm._memory.shape[1]

        map_row = lambda row: ''.join('@' if x else '.' for x in row)

        map_content = '\n'.join(map_row(row) for row in self._gm._memory)
        map_file_content = f"type octile\nheight {height}\nwidth {width}\nmap\n{map_content}"

        return map_file_content

    def format_task_sring(self, start_xy, target_xy, map_shape=None):
        if map_shape is None:
            map_shape = self._gm._memory.shape
        task_file_content = f"{self.idx}	tmp.map	{map_shape[0]}	{map_shape[1]}	"
        task_file_content += f"{start_xy[1]}	{start_xy[0]}	{target_xy[1]}	{target_xy[0]}	1\n"
        return task_file_content

    def get_task_content(self, xy, target_xy):
        offset = self._gm._memory.shape[0] // 2
        start_xy = (xy[0] + offset, xy[1] + offset)
        target_xy = (target_xy[0] + offset, target_xy[1] + offset)
    
        task_file_content = self.format_task_sring(start_xy, target_xy)
        return task_file_content
    
    def get_task_file(self, xy, target_xy):
        task_file_content = "version 1\n"
        task_file_content += self.get_task_content(xy, target_xy)
        return task_file_content

    def parse_solution_data(self, file_path):
        offset = self._gm._memory.shape[0] // 2
        with open(file_path) as f:
            file_content = f.read()
        
        lines = file_content.split('\n')
        
        start_index = next(i for i, line in enumerate(lines) if line.startswith('solution='))
        moves = []
        for line in lines[start_index+1:]:
            if line.strip() == "":
                break
            coords = line.split(':')[1].strip('(),')
            x, y = map(int, coords.split(','))
            moves.append((y-offset, x-offset))
        
        return moves
    
    def run_lacam(self, map_file_content, scene_file_content, num_agents):
        lacam_results = self.lacam_lib.run_lacam(map_file_content, scene_file_content, num_agents)

        lines = lacam_results.strip().split('\n')
        target_path = []
        for line in lines:
            tuples = [tuple(map(int, item.split(','))) for item in line.strip().split('|') if item]
            assert len(tuples) == 1
            target_path.append(tuples[0][::-1]) # invert (x,y) to (y,x)
        
        return target_path

    def find_lacam_path(self, xy, target_xy):
        task_content = self.get_task_file(xy, target_xy)
        map_content = self.get_map_file()

        target_path = self.run_lacam(map_content, task_content, 1)
        return target_path

    def act(self, obs, use_astar=False):
        xy, target_xy, obstacles, agents = obs['xy'], obs['target_xy'], obs['obstacles'], obs['agents']

        if self._saved_xy is not None and h(self._saved_xy, xy) > 1:
            raise IndexError("Agent moved more than 1 step. Please, call clear_state method before new episode.")
        if not self.global_planning and self._saved_xy is not None and h(self._saved_xy, xy) == 0 and xy != target_xy:
            return self._rnd.integers(len(self._moves))
        
        self._gm.update(*xy, obstacles)

        if use_astar:
            path = a_star(xy, target_xy, self._gm)
        else:
            if self.global_planning:
                assert len(self.calculated_path) > 0
                path = self.calculated_path[::-1]
            else:
                path = self.find_lacam_path(xy, target_xy)

        if len(path) <= 1:
            action = 0
        else:
            (x, y), (tx, ty), *_ = path
            action = self._reverse_actions[tx - x, ty - y]
            if self.global_planning:
                # delete the first node from the precalculated path (last - because inversed)
                self.calculated_path.pop()
        
        self._saved_xy = xy
        return action

    def clear_state(self):
        self._saved_xy = None
        self._gm = GridMemory()


class BatchLacamAgent:
    class Planner_Mode(Enum):
        A_STAR = "a_star"
        DECENTRALIZED_LACAM = "decentralized_lacam"
        CENTRALIZED_LACAM = "centralized_lacam"

    def __init__(self, lacam_lib_path="lacam3/build/liblacam.so", mode=Planner_Mode.CENTRALIZED_LACAM):
        assert isinstance(mode, BatchLacamAgent.Planner_Mode)

        self.lacam_agents = {}
        self.lacam_lib = LacamLib(lacam_lib_path)
        self.steps = 0
        self.mode = mode

    def parse_data(self, data):
        lines = data.strip().split('\n')
        columns = None

        for line in lines:
            tuples = [tuple(map(int, item.split(','))) for item in line.strip().split('|') if item]
            if columns is None:
                columns = [[] for _ in range(len(tuples))]
            for i, t in enumerate(tuples):
                columns[i].append(t[::-1])
        
        return columns

    def act(self, observations, env):
        self.steps += 1
        use_astar = self.mode == BatchLacamAgent.Planner_Mode.A_STAR
        global_planning = self.mode == BatchLacamAgent.Planner_Mode.CENTRALIZED_LACAM

        actions = []
        if self.mode == BatchLacamAgent.Planner_Mode.CENTRALIZED_LACAM:
            map_array = env.grid.obstacles
            agent_starts_xy = env.grid.get_agents_xy()
            agent_targets_xy = env.grid.get_targets_xy()

            has_new_tasks = False
            task_file_content = "version 1\n"
            for idx, (start_xy, target_xy) in enumerate(zip(agent_starts_xy, agent_targets_xy)):
                if idx not in self.lacam_agents:
                    self.lacam_agents[idx] = LacamAgent(idx, self.lacam_lib, global_planning=global_planning)
                
                if self.lacam_agents[idx].is_new_goal(target_xy):
                    self.lacam_agents[idx].set_new_goal(target_xy)
                    has_new_tasks = True
                agent_task = self.lacam_agents[idx].format_task_sring(start_xy, target_xy, map_shape=map_array.shape)
                task_file_content += agent_task
            
            if has_new_tasks:
                print(f'Step {self.steps}: generating new plan')
                map_row = lambda row: ''.join('@' if x else '.' for x in row)
                map_content = '\n'.join(map_row(row) for row in map_array)
                map_file_content = f"type octile\nheight {map_array.shape[0]}\nwidth {map_array.shape[1]}\nmap\n{map_content}"
                lacam_results = self.lacam_lib.run_lacam(map_file_content, task_file_content, len(self.lacam_agents))
                agent_paths = self.parse_data(lacam_results)
                for idx, agent_path in enumerate(agent_paths):
                    self.lacam_agents[idx].set_calculated_path(agent_path)

        for idx, obs in enumerate(observations):
            if idx not in self.lacam_agents:
                self.lacam_agents[idx] = LacamAgent(idx, self.lacam_lib, global_planning=global_planning)
            actions.append(self.lacam_agents[idx].act(
                obs, use_astar=use_astar)
            )
        
        return actions

    def reset_states(self):
        self.lacam_agents = {}

In [20]:
def run_planner(env, mode, image_prefix='exp1'):
    print(f'Running {mode.value} planner')
    obs, info = env.reset()
    agent = BatchLacamAgent(lacam_lib_path="lacam3/build/liblacam.so", mode=mode)

    time_start = time.time()
    while True:
        actions_agent = agent.act(obs, env)
        obs, reward, terminated, truncated, info = env.step(actions_agent)
        if all(terminated) or all(truncated):
            break

    time_end = time.time()
    print(f'Time elapsed: {round(time_end - time_start, 1)}s')

    env.save_animation(f"{image_prefix}_{mode.value}.svg", AnimationConfig(egocentric_idx=0))
    print(f'Saved result to "{image_prefix}_{mode.value}".svg')

def display_svgs_in_row(svgs):
    svg_displays = [f'<div style="display: inline-block; text-align: center; margin-right: 20px;">'
                    f'<p>{label}</p>'
                    f'{SVG(filename).data}'
                    f'</div>'
                    for label, filename in svgs]
    display_html = '<div style="display: flex; align-items: center;">' + ''.join(svg_displays) + '</div>'
    display(HTML(display_html))

In [16]:
NUM_AGENTS = 8
MAX_STEPS = 200
grid_config = GridConfig(num_agents=NUM_AGENTS, size=15, density=0.3, seed=1, max_episode_steps=MAX_STEPS, obs_radius=3,
                         observation_type='POMAPF', collission_system='soft', on_target='restart')

env = pogema_v0(grid_config)
env = AnimationMonitor(env)

run_planner(env, mode=BatchLacamAgent.Planner_Mode.A_STAR, image_prefix='exp1')
run_planner(env, mode=BatchLacamAgent.Planner_Mode.DECENTRALIZED_LACAM, image_prefix='exp1')
run_planner(env, mode=BatchLacamAgent.Planner_Mode.CENTRALIZED_LACAM, image_prefix='exp1')

Running a_star planner
Time elapsed: 0.3s
Saved result to "a_star".svg
Running decentralized_lacam planner
Time elapsed: 17.6s
Saved result to "decentralized_lacam".svg
Running centralized_lacam planner
Step 1: generating new plan
Time elapsed: 3.1s
Saved result to "centralized_lacam".svg


In [18]:
svgs = [
    ("AStar", 'exp1_a_star.svg'),
    ("Decentralized Lacam", 'exp1_decentralized_lacam.svg'),
    ("Centralized Lacam", 'exp1_centralized_lacam.svg')
]

display_svgs_in_row(svgs)

In [ ]:
NUM_AGENTS = 30
MAX_STEPS = 100
grid_config = GridConfig(num_agents=NUM_AGENTS, size=25, density=0.3, seed=1, max_episode_steps=MAX_STEPS, obs_radius=3,
                         observation_type='POMAPF', collission_system='soft', on_target='restart')
                        #  observation_type='POMAPF', collission_system='soft', on_target='finish')

env = pogema_v0(grid_config)
env = AnimationMonitor(env)

run_planner(env, mode=BatchLacamAgent.Planner_Mode.A_STAR, image_prefix='exp1')
run_planner(env, mode=BatchLacamAgent.Planner_Mode.DECENTRALIZED_LACAM, image_prefix='exp1')
run_planner(env, mode=BatchLacamAgent.Planner_Mode.CENTRALIZED_LACAM, image_prefix='exp1')